# Rockpool

Rockpool is a Python package for working with dynamical neural network architectures, particularly for designing event-driven networks for Neuromorphic computing hardware. Rockpool provides a convenient interface for designing, training and evaluating recurrent networks, which can operate both with continuous-time dynamics and event-driven dynamics.

You can import a model from NIR with the `from_nir` method in `rockpool.nn.modules.torch.nir`. Once it is in Rockpool, you deploy your model to Synsense's Xylo chips.

## Import a NIR graph to Rockpool

In [ ]:
import torch
from rockpool.nn.modules.torch.nir import from_nir

import nir

# Supress warnings
import warnings
# warnings.filterwarnings('ignore')

# Create a NIR graph
affine_weights = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
affine_bias = torch.tensor([1.0, 2.0])
li_tau = torch.tensor([0.9, 0.8])
li_r = torch.tensor([1.0, 1.0])
li_v_leak = torch.tensor([0.0, 0.0])
nir_network = nir.NIRGraph.from_list(
    nir.Affine(affine_weights, affine_bias), nir.LI(li_tau, li_r, li_v_leak)
)

# Create Rockpool model from NIR graph.
with warnings.catch_warnings():
    # Supress unrelated warnings from Rockpool
    warnings.simplefilter("ignore")
    rockpool_model = from_nir(nir_network)
print(rockpool_model)


## Export a NIR graph from Rockpool

In [ ]:
from rockpool.nn.modules.torch.nir import to_nir
from rockpool.nn.modules import LinearTorch, LIFTorch
from rockpool.nn.combinators import Sequential

n_in = 2
n_hidden = 4
n_out = 2
dt = 1e-3

# Create Rockpool model
net = Sequential(
    LinearTorch((n_in, n_hidden), has_bias=False),
    LIFTorch(n_hidden, dt=dt),
    LinearTorch((n_hidden, n_out), has_bias=False),
    LIFTorch(n_out, dt=dt),
)

# Convert model to NIR graph with a random input of representative shape
nir_graph = to_nir(net, torch.randn(1, 2))
print(nir_graph)


# Reload sinabs model from NIR
rockpool_model = from_nir(nir_graph)
